In [1]:
import pandas as pd
import json

In [2]:
# Membaca file CSV ke dalam pandas DataFrame
import os
os.chdir('C:/Users/Lenovo/Skripsi/Notebook')

df = pd.read_csv('output_keluar_masuk_asean_all.csv')

In [3]:
print(df.head())

        mmsi                    Port        dt_pos_utc     fc_vessel  \
0  518998571            Port Dickson    10/4/2022 9:50  Cook Islands   
1  518998571            Port Dickson  10/12/2022 11:35  Cook Islands   
2  518998571                     NaN  10/12/2022 13:08  Cook Islands   
3  518998571                     NaN  10/12/2022 14:20  Cook Islands   
4  518998571  Pelabuhan Sungai Udang  10/12/2022 14:31  Cook Islands   

  sc_vessel vessel_type         ns_vessel  draught  position  latitude  \
0     Asing      Tanker         At Anchor      5.2   in port  2.390083   
1     Asing      Tanker         At Anchor      5.2   in port  2.389017   
2     Asing      Tanker  Underway Sailing      5.2  out port  2.291333   
3     Asing      Tanker  Underway Sailing      5.0  out port  2.185033   
4     Asing      Tanker  Underway Sailing      5.0   in port  2.171383   

    longitude masuk_pelabuhan keluar_pelabuhan  \
0  101.884383               -                -   
1  101.886650         

In [4]:
# Cetak nama kolom
print(df.columns)

Index(['mmsi', 'Port', 'dt_pos_utc', 'fc_vessel', 'sc_vessel', 'vessel_type',
       'ns_vessel', 'draught', 'position', 'latitude', 'longitude',
       'masuk_pelabuhan', 'keluar_pelabuhan', 'geojson'],
      dtype='object')


In [5]:
# Cetak record pertama (indeks 0) dari kolom terakhir
record_pertama_kolom_terakhir = df.iloc[0, -1]
print(record_pertama_kolom_terakhir)

{"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {"time": "10/04/2022"}, "geometry": {"type": "LineString", "coordinates": [[101.8843833, 2.39008333, 0, 1664851800]]}}]}


In [7]:
# Fungsi untuk menggabungkan koordinat dari geojson
def gabung_koordinat(geojson_list):
    all_coordinates = []
    for geojson_str in geojson_list:
        geojson = json.loads(geojson_str)
        coordinates = geojson['features'][0]['geometry']['coordinates']
        all_coordinates.extend(coordinates)
    # Buat geojson baru dengan koordinat yang telah digabungkan
    combined_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {},
                "geometry": {
                    "type": "LineString",
                    "coordinates": all_coordinates
                }
            }
        ]
    }
    return json.dumps(combined_geojson)

# Kelompokkan data berdasarkan 'mmsi' dan gabungkan koordinat
df_grouped = df.groupby('mmsi').agg({
    'geojson': gabung_koordinat,
    'ns_vessel': 'first',
    'dt_pos_utc': 'first',
    'vessel_type': 'first',
    'fc_vessel': 'first',
    'draught': 'first',
    'latitude': 'first',
    'longitude': 'first',
#     'sog': 'first',
#     'H3_int_index_8': 'first'
#     'masuk_indo': 'first',
#     'keluar_indo': 'first'
}).reset_index()

# Cetak DataFrame baru


In [8]:
print(df_grouped)

         mmsi                                            geojson  \
0   477716400  {"type": "FeatureCollection", "features": [{"t...   
1   477717300  {"type": "FeatureCollection", "features": [{"t...   
2   477749600  {"type": "FeatureCollection", "features": [{"t...   
3   477767900  {"type": "FeatureCollection", "features": [{"t...   
4   477779200  {"type": "FeatureCollection", "features": [{"t...   
..        ...                                                ...   
95  565156000  {"type": "FeatureCollection", "features": [{"t...   
96  565390000  {"type": "FeatureCollection", "features": [{"t...   
97  565585000  {"type": "FeatureCollection", "features": [{"t...   
98  565943000  {"type": "FeatureCollection", "features": [{"t...   
99  566134000  {"type": "FeatureCollection", "features": [{"t...   

                 ns_vessel       dt_pos_utc vessel_type  fc_vessel  draught  \
0         Underway Sailing   10/4/2022 9:10       Cargo  Hong Kong     12.7   
1   Under Way Using Engin

In [9]:
# Cetak nama kolom
print(df_grouped.columns)

Index(['mmsi', 'geojson', 'ns_vessel', 'dt_pos_utc', 'vessel_type',
       'fc_vessel', 'draught', 'latitude', 'longitude'],
      dtype='object')


In [10]:
df_mmsi_json = df_grouped[['mmsi', 'vessel_type', 'geojson','dt_pos_utc']]

In [11]:
print(df_mmsi_json)

         mmsi vessel_type                                            geojson  \
0   477716400       Cargo  {"type": "FeatureCollection", "features": [{"t...   
1   477717300       Cargo  {"type": "FeatureCollection", "features": [{"t...   
2   477749600       Cargo  {"type": "FeatureCollection", "features": [{"t...   
3   477767900       Cargo  {"type": "FeatureCollection", "features": [{"t...   
4   477779200      Tanker  {"type": "FeatureCollection", "features": [{"t...   
..        ...         ...                                                ...   
95  565156000       Cargo  {"type": "FeatureCollection", "features": [{"t...   
96  565390000      Tanker  {"type": "FeatureCollection", "features": [{"t...   
97  565585000      Tanker  {"type": "FeatureCollection", "features": [{"t...   
98  565943000      Tanker  {"type": "FeatureCollection", "features": [{"t...   
99  566134000      Tanker  {"type": "FeatureCollection", "features": [{"t...   

         dt_pos_utc  
0    10/4/2022 9:

In [12]:
# Cetak record pertama (indeks 0) dari kolom terakhir
record_pertama_kolom_terakhir = df_mmsi_json.iloc[0, -1]
print(record_pertama_kolom_terakhir)

10/4/2022 9:10


In [13]:
df_mmsi_json.to_csv("grouped_keluar_masuk_asean_all.csv", index=False)